In [3]:
import numpy as np
import tensorflow as tf

# %matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors as colors 

import ipysh

%aimport models.base

import Hunch_utils  as Htls
import Hunch_lsplot as Hplt
import Hunch_tSNEplot as Hsne

%aimport models.AEFIT4
%aimport models.AEFIT5
%aimport models.Compose

# ipysh.Bootstrap_support.debug()

In [4]:
import Dummy_g1data as dummy
du1 = dummy.Dummy_g1data(counts=40000, size=15).buffer()
du2 = dummy.Dummy_g1data(counts=40000, size=20)

In [15]:
m1 = models.AEFIT5.AEFIT5(latent_dim=2, feature_dim=30,  dprate=0., scale=1, beta=0., name='m1')
m2 = models.AEFIT5.AEFIT5(latent_dim=2, feature_dim=30,  dprate=0., scale=1, beta=0., name='m2')
hm = models.AEFIT5.AEFIT5(latent_dim=2, feature_dim=4, beta=0., name='hidden')
# 
# m2.compile(
#     optimizer= tf.keras.optimizers.Adam(learning_rate=1e-3),
#     loss = tf.keras.losses.mean_squared_error
# )

AEFIT5 ready:
AEFIT5 ready:
AEFIT5 ready:


In [16]:
d = du1.ds_array.batch(100).map(lambda x,y: (x,x) )
m1.fit(d, epochs=1)

400/400 [==============================] - 10s 26ms/step - loss: 14.7527 - accuracy: 0.6872


In [17]:
d = du1.ds_array.batch(100).map(lambda x,y: (x,x) )
m2.fit(d, epochs=1)

400/400 [==============================] - 10s 26ms/step - loss: 14.7993 - accuracy: 0.6827


In [18]:
h2 = models.Compose.Compose2().set_model(hm).compose([m1,m2])
h2._model.bypass = True
m1.trainable = False
m2.trainable = False
hm.trainable = True
h2.compile()

In [19]:
[ v.name for v in h2.trainable_variables ]

['na_n_dense_5/kernel:0',
 'na_n_dense_5/bias:0',
 'dense_55/kernel:0',
 'dense_55/bias:0',
 'dense_56/kernel:0',
 'dense_56/bias:0',
 'dense_57/kernel:0',
 'dense_57/bias:0',
 'dense_58/kernel:0',
 'dense_58/bias:0',
 'dense_59/kernel:0',
 'dense_59/bias:0',
 'dense_60/kernel:0',
 'dense_60/bias:0',
 'dense_61/kernel:0',
 'dense_61/bias:0',
 'dense_62/kernel:0',
 'dense_62/bias:0',
 'dense_63/kernel:0',
 'dense_63/bias:0',
 'dense_64/kernel:0',
 'dense_64/bias:0',
 'dense_65/kernel:0',
 'dense_65/bias:0']

In [20]:
d = du1.ds_array.batch(100).map(lambda x,y: ((x,x),(x,x)) )
h2.fit(d, epochs=1)

400/400 [==============================] - 12s 30ms/step - loss: 36.2607 - model_2_loss: 17.5681 - model_2_1_loss: 18.6925


In [12]:
p = Hplt.LSPlotBokeh()
p.set_model(m1)
p.set_data(du1, counts=1000)
p.plot(notebook_url='http://172.17.0.2:8888')

Loading BokehJS ...

In [ ]:
hm = models.AEFIT5.AEFIT5(latent_dim=2, feature_dim=20, beta=0.)
m1 = models.AEFIT5.AEFIT5(latent_dim=10, feature_dim=30,  dprate=0., scale=1, beta=0.)
m2 = models.AEFIT5.AEFIT5(latent_dim=10, feature_dim=40,  dprate=0., scale=1, beta=0.)

In [ ]:
h = models.Compose.Compose().set_model(hm).compose([m1,m2])

In [ ]:
def get_tf_dataset(ds1, ds2):
    types = (tf.float32, tf.int32), (tf.float32, tf.int32)
    shape = ((2*ds1.dim,),()),((2*ds2.dim,),())
    def gen():
        import itertools
        for i in itertools.count(0):
            if i < min(len(ds1),len(ds2)):
                s1,l1 = ds1.gen_pt(i)
                s2,l2 = ds2.gen_pt(i)
                yield (np.concatenate([s1[:,0], s1[:,1]]), l1), (np.concatenate([s2[:,0], s2[:,1]]), l2)
            else:
                return
    return tf.data.Dataset.from_generator(gen, types, shape)

                       
ds = get_tf_dataset(du1,du2)
ds = ds.batch(100).map(lambda x,y: ((x[0],y[0]),(x[0],y[0])) )
ds

# h.fit(d, epochs=5)

In [ ]:
h.fit(ds, epochs=5)

In [ ]:
#
